## import libraries

In [1]:
import asyncio
import math
from math import log, sin, cos, tan, exp, sqrt, pi
import time
from random import randrange
import torch
import numpy as np
from classroom import Classroom
from classroom import Student
from classroom import BytesDataset
from classroom import GutenbergSnippetsDataset
from classroom import GutenbergBitsDataset
from classroom import MLPLM, MyLM, ABPCNLM
from classroom import TransformerLM
from classroom import AdamW
from classroom import Sonny
from classroom import Floyd
from classroom import Plot
from classroom import Fun
from classroom import Count
from classroom import Sum
from classroom import Diff
from classroom import Log2Sum
from classroom import KalmanFilter1D
from classroom import MedianFilter
from classroom import TwoWindowFilter
from classroom import numel
from classroom import utf8decode, utf8encode
from pathlib import Path
import numba

Loading BokehJS ...

## initialize model

In [2]:
if True:
    path = '2021-10-03-2008.pt'
    model = torch.load(path).to('cuda')

In [ ]:
if True:
    model = (
        ABPCNLM(
            n_vocab_in=2,
            n_ctx=2**14,
            d_model=1,
            n_layers=2,
            d_hidden=1024,
            nonlinearity="GELU",
            p_dropout=0.0,
            n_vocab_out=2).to('cuda'))

In [ ]:
numel(model), numel(model)*4/1E9

In [ ]:
numel(model)*3*2.09*2048/1E12

## initialize student

In [3]:
optimizer = AdamW(parameters=model.named_parameters())
dataset = GutenbergBitsDataset()
batch_size = None
example_length = model.n_ctx + 1
student= Student(
    model=model,
    optimizer=optimizer,
    dataset=dataset,
    batch_size=batch_size,
    example_length=example_length)

## schedule hyperparameters

In [11]:
student.batch_size=1024
for (idx, (pn, p)) in enumerate(student.model.named_parameters()):
    batch_multiplier = 1
    lr_base = 1e-8
    lr = lambda n: lr_base*(n%1000)/1000
    student.optimizer.state[pn]["lr"]           = lambda n: lr(n)
    student.optimizer.state[pn]["beta1"]        = lambda n: 0.9
    student.optimizer.state[pn]["beta2"]        = lambda n: 0.999
    student.optimizer.state[pn]["weight_decay"] = lambda n: 0.001
    student.optimizer.state[pn]["update"]       = lambda n: n%batch_multiplier == 0

## initialize baseline

In [5]:
student.reset_baseline()

## start training

In [6]:
classroom = Classroom()

In [7]:
classroom.enroll(student)

## autocomplete

In [ ]:
def autocomplete(prompt=None):
    if prompt is None:
        prompt = student.dataset.decode(student.dataset.batch(1, 768).tolist()[0])
    print(student.autocomplete(prompt=prompt, n_generate=1024, temp=1.0))
autocomplete()

## plots

In [ ]:
import time
plot_data = {}
lag = 5000
for (idx, student) in enumerate(classroom.students):
    X = Fun(Count(), student.times)
    Y = Fun(TwoWindowFilter(lag=lag), student.grades)
    Z = Fun(TwoWindowFilter(lag=lag), student.baseline_grades)
    plot_data.update({f"grades-{idx}": (X, Y)})
    plot_data.update({f"baseline-{idx}": (X, Z)})
Plot(**plot_data)

In [12]:
import time
plot_data_2 = {}
lag = 500
for (idx, student) in enumerate(classroom.students):
    X = Fun(Count(), student.times)
    Y = Fun(lambda x, y: x - y, student.grades, student.baseline_grades)
    Y = Fun(TwoWindowFilter(lag=lag), Y.output, aux=Y)
    plot_data_2.update({f"improvement-{idx}": (X, Y)})
Plot(**plot_data_2)

## stats

In [13]:
for (idx, student) in enumerate(classroom.students):
    print(f"\nStudent #{idx}\n==========")
    N = 50
    n = len(student.times)-1
    time = student.time
    mean_grade = np.mean(np.array(student.grades[n-N:n]))
    mean_baseline_grade = np.mean(np.array(student.baseline_grades[n-N:n]))
    mean_improvement = mean_grade - mean_baseline_grade
    improvement_rate = mean_improvement / (time - student.time_of_last_baseline)
    message = '\n'.join([
        f"batch_size            = {student.batch_size}",
        f"example_length        = {student.example_length}",
        f"n                     = {n}",
        f"time                  = {int(time)}s",
        f"time_of_last_baseline = {int(student.time_of_last_baseline)}s",
        f"steps per second      = {(n/time)}",
        f"mean_baseline_grade   = {mean_baseline_grade}",
        f"mean_grade            = {mean_grade}",
        f"mean_improvement      = {mean_improvement}",
        f"improvement_rate      = {improvement_rate} per second",
        f"time to 75            = {(.75-mean_grade)/improvement_rate/3600} hours",
    ])
    print(message)


Student #0
batch_size            = 1024
example_length        = 16385
n                     = 856
time                  = 581s
time_of_last_baseline = 0s
steps per second      = 1.4718607115229054
mean_baseline_grade   = 0.49461800754070284
mean_grade            = 0.4959204560518265
mean_improvement      = 0.001302448511123655
improvement_rate      = 2.239512607832257e-06 per second
time to 75            = 31.514737112835775 hours


## save

In [14]:
torch.save(student.model, f='2021-10-03-2008.pt')

In [ ]:
import asyncio
async def autosave():
    while True:
        await asyncio.sleep(3600)
        torch.save(student.model, f='2021-10-03-2008.auto.pt')
task = asyncio.create_task(autosave())


In [ ]:
lyles_constant = 9115131782/14818489608 #* log(50257)/log(65536)
lyles_constant

In [ ]:
mg = 0.792844

In [ ]:
(1 - mg) / lyles_constant * log(50257)  # gpt2natscale (for kaplan paper comparison)

In [ ]:
(1 - mg) * 8  #  bits per character

In [ ]:
2**((1 - mg) * 8)  # perplexity

## parameter histograms

In [ ]:
histograms = []
for (idx, (pn, p)) in enumerate(student.model.named_parameters()):
    with torch.no_grad():
        print(idx, pn, torch.sqrt(torch.var(p)).item())
        Y, X = np.histogram(p.detach().cpu().numpy(), bins=int(sqrt(torch.numel(p))), density=True)
        print(X.shape, Y.shape)
        histograms.append(Plot(**{f"hist-{idx}": (X.tolist(), Y.tolist())}))

In [ ]:
histograms[4] # 3 7 9 13 15 21 43

## batch-level grade histogram

In [ ]:
Y, X = np.histogram(student.grades[-5000:], bins=256, range=(0,1.0), density=True)
V, U = np.histogram(student.baseline_grades[-5000:], bins=256, range=(0,1.0), density=True)
Plot(**{f"grade-hist": (X, Y), "baseline": (U, V)})

In [ ]:
model.n_ctx, model.d_model, model.d_hidden, model.n_layers

## example-level grade histogram

In [ ]:
def get_graded_examples():
    result = []
    for batch_idx in range(16):
        print(f"batch_idx = {batch_idx}/256")
        x = student.dataset.batch(student.batch_size, student.example_length)
        print(f"orig {x.shape}")
        with torch.no_grad():
            y = student.model(x)
            y = 1.0 - y.cpu().numpy()
            result.append(y)
    data = np.concatenate(result, axis=0)
    result = data.tolist()
    return result

In [ ]:
graded_examples = get_graded_examples()

In [ ]:
len(graded_examples)

In [ ]:
sum(x for x in graded_examples)/len(graded_examples)

In [ ]:
R = (0, 1)
def XYFor(k):
    es = graded_examples
    bins = int(sqrt(len(es)))
    Y, X = np.histogram(es, bins=bins, range=R, density=True)
    return (X, Y)
Plot(**{f"examples-hist-{k}": XYFor(k) for k in [1]})

In [ ]:
ord(' ')

In [ ]:
np.mean(example_grades)

In [ ]:
(1 - 0.7870894884999871)*8

In [ ]:
(1 - 0.8)*8

In [ ]:
(1 - 0.9)*8

In [ ]:
x = np.array([[1,2],[3,4]],dtype=np.uint8)

In [ ]:
np.unpackbits(x, axis=1)